# **Text classification with pretrained word embeddings (GLOve)**

 

In this example, you will show how to train a text classification model that uses pre-trained word embeddings.

## Importing the libraries and setting up

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Downloading the Newsgroup20 data

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

## The data information

In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['rec.autos', 'talk.politics.misc', 'talk.politics.mideast', 'sci.med', 'comp.sys.mac.hardware', 'talk.religion.misc', 'alt.atheism', 'comp.sys.ibm.pc.hardware', 'sci.crypt', 'misc.forsale', 'comp.windows.x', 'rec.sport.hockey', 'sci.space', 'rec.motorcycles', 'talk.politics.guns', 'sci.electronics', 'soc.religion.christian', 'comp.graphics', 'comp.os.ms-windows.misc', 'rec.sport.baseball']
Number of files in comp.graphics: 1000
Some example filenames: ['38388', '39045', '38847', '38512', '38333']


An example of what one file contains:

In [4]:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



In [5]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

**Because of crashing the code, I had to use just 15000 for samples and labels.**

In [6]:
print(len(samples))
samples = samples[:15000]

19997


In [7]:
print(len(labels))
labels = labels[:15000]

19997


## Shuffle and split the data into training & validation sets

In [8]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [9]:

print(num_validation_samples)
print(len(train_samples))
print(len(val_samples))
print(len(train_labels))
print(len(val_labels))

3000
12000
3000
12000
3000


## Create a vocabulary index by using the TextVectorization 

In [10]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [11]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'a']

In [12]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([    2,  2778,  1642,    14,     2, 10297])

In [13]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [14]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 2778, 1642, 14, 2, 10297]

## Downloading pre-trained GloVe embeddings (a 822M zip file)
I tried to just upload one file in this set and upload it in the dirve, because of lack of space

In [15]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [16]:
#!unzip -q glove.6B.zip

## Importing glove.6B.100d.txt file

In [17]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '11TGu1qTAubTcwCVEInQup7mFD_xdHjud'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('glove.6B.100d.txt')



In [18]:
# path_to_glove_file = os.path.join(
#     os.path.expanduser("/"), "content/glove.6B.100d.txt"
    
# )
path_to_glove_file = "/content/glove.6B.100d.txt"


embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [19]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 17328 words (2672 misses)


In [20]:
#voc

In [21]:
num_tokens

20002

In [22]:
embedding_dim

100

In [23]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## Building the model


In [24]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)        

In [25]:
len(train_samples)

12000

In [26]:
#train_samples = train_samples[:1000]

In [27]:
len(train_labels)

12000

In [28]:
#train_labels = train_labels[:1000]

In [29]:
len(val_labels)

3000

In [30]:
#val_labels = val_labels[:250]

# **Train the model**


In [31]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()

In [32]:
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

In [33]:
y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [34]:
print(len(y_train))
print(len(y_val))

12000
3000


We use categorical crossentropy as our loss since we're doing softmax classification.
Moreover, we use `sparse_categorical_crossentropy` since our labels are integers.

In [35]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
94/94 [==============================] - 1s 13ms/step - loss: 2.5578 - acc: 0.1557 - val_loss: 2.0329 - val_acc: 0.3297
Epoch 2/20
94/94 [==============================] - 1s 9ms/step - loss: 1.8700 - acc: 0.3381 - val_loss: 1.4259 - val_acc: 0.5130
Epoch 3/20
94/94 [==============================] - 1s 9ms/step - loss: 1.4033 - acc: 0.5205 - val_loss: 1.4934 - val_acc: 0.4860
Epoch 4/20
94/94 [==============================] - 1s 9ms/step - loss: 1.1544 - acc: 0.6075 - val_loss: 1.0719 - val_acc: 0.6170
Epoch 5/20
94/94 [==============================] - 1s 9ms/step - loss: 0.9755 - acc: 0.6671 - val_loss: 0.9795 - val_acc: 0.6737
Epoch 6/20
94/94 [==============================] - 1s 9ms/step - loss: 0.8500 - acc: 0.7102 - val_loss: 0.9020 - val_acc: 0.6973
Epoch 7/20
94/94 [==============================] - 1s 9ms/step - loss: 0.7353 - acc: 0.7496 - val_loss: 0.9152 - val_acc: 0.6993
Epoch 8/20
94/94 [==============================] - 1s 9ms/step - loss: 0.6410 - acc: 0.7

## Exporting an end-to-end model


In [36]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

'comp.graphics'

In [37]:
class_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [38]:
probabilities

array([[2.0399250e-11, 9.9998605e-01, 4.2076911e-07, 9.6021795e-09,
        5.6871363e-10, 1.2978877e-05, 1.9878927e-09, 1.2809371e-12,
        3.0314376e-11, 1.6657487e-14, 8.8235166e-12, 1.3958975e-11,
        2.8750358e-09, 3.3864575e-07, 1.5065353e-07, 1.3916048e-29,
        5.5010014e-31, 2.1156786e-30, 5.3371487e-30, 8.1624312e-30]],
      dtype=float32)